In [ ]:
import json
import os
import fnmatch
import numpy as np
from collections import defaultdict

In [ ]:
rosters = []
for YEAR in range(1952,2020):

    rosters.append(json.load(open('rosters/auto_roster_{}.json'.format(YEAR))))


In [ ]:
nc2 = np.array([5,1,4,2,1,1,2,1,1,3,7,3,3,3,1])/38
y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }
inv_ymap = {v:k for k,v in y_map.items()}
y_keys = ['hgt',
 'str',
 'spd',
 'jmp',
 'end',
 'ins',
 'dnk',
 'ft.1',
 '2pt',
 '3pt',
 'oiq',
 'diq',
 'drb',
 'pss',
 'reb']

In [ ]:
players = []
for roster in rosters:
    yr = roster['startingSeason']
    teams = {t['tid']:t for t in roster['teams']}
    for t in teams:
        teams[t]['plist'] = []
        teams[t]['year'] = yr
    for player in roster['players']:
        tid = player['tid']
        prate = player['ratings'][0]
        #print(prate)
        r = np.array([prate[inv_ymap[k]] for k in y_keys])
        ovr = (r * nc2).sum()
        if tid in teams:
            teams[tid]['plist'].append((ovr,player))
    players += list(teams.values())

In [ ]:
#prate = players[0]['plist'][0]['ratings'][0]
#r = np.array([prate[inv_ymap[k]] for k in y_keys])
#prate
#y_map
for player in players:
    wtotal = 0
    rt = defaultdict(float)
    pidx = [(op[0],i) for i,op in enumerate(player['plist'])]
    plist = [p for o,p in player['plist']]

    for i,pi in enumerate(sorted(pidx,reverse=True)):
        p = plist[pi[1]]
        w= (1/(i+1))**1.6
        for r,rv in p['ratings'][0].items():
            rt[r] += w*rv
        rt['HEIGHT'] += w*p['hgt']
        rt['WEIGHT'] += w*p['weight']
        rt['AGE'] += w*(player['year']- p['born']['year'])

        wtotal += w
    for k,v in rt.items():
        rt[k] = int(round(v/wtotal))
    player['hgt'] = int(round(rt['HEIGHT']))
    player['weight'] = int(round(rt['WEIGHT']))
    player['born'] = {'loc':'','year':int(round(2019-rt['AGE']))}
    del rt['AGE']
    del rt['HEIGHT']
    del rt['WEIGHT']
    del player['plist']
    player['ratings'] = [dict(rt)]
    player['name'] = player['region'] + ' ' + player['name'] + str(player['year'])
    del player['region']
    del player['year']
    del player['cid']
    del player['did']
    del player['pop']
    del player['stadiumCapacity']
    del player['tid']
    del player['strategy']




In [ ]:
player

In [ ]:
new_team = [
        ('PHX','PHO'),
        ('BKN','BRK'),
        ('CHA','CHO'),
        ('NOP','NOH'),
        ('NOH','NOK'),
        ('BRK','NJN'),
        ('OKC','SEA'),
        ('MEM','VAN'),
        ('WAS','WSB'),
        ('SAC','KCK'),
        ('LAC','SDC'),
        ('UTA','NOJ'),
        ('SDC','BUF'),
        ('NJN','NYN'),
        ('NYN','NYA'),
        ('NYA','NJA'),
        ('GSW','SFW'),
        ('SAS','DLC'),
        ('DLC','TEX'),
        ('HOU','SDR'),
        ('CHA','CHH'),
        ('SAS','SAA'),
        ('SAA','TEX'),
        ('DEN','DNA'),
        ('DNA','DNR'),
        ('WSB','CAP'),
        ('CAP','BAL'),
        ('BAL','CHZ'),
        ('CHZ','CHP'),
        ('NOH','KEN'),
        ('MIN','SDS'),
        ('SDS','SDA'),
        ('MIA','FLO'),
        ('FLO','MMF'),
        ('MMF','MNM'),
        ('SFW','PHW'),
        ('LAL','MNL'),
        ('UTA','UTS'),
        ('UTS','LAS'),
        ('LAS','ANA'),
        ('CHH','SSL'),
        ('SSL','CAR'),
        ('CAR','HSM'),
        ('DET','FTW'),
        ('ATL','STL'),
        ('STL','MLH'),
        ('MLH','TRI'),
        ('PHI','SYR'),
        ('TOR','PTC'),
        ('PTC','PTP'),
        ('PTP','MNP'),
        ('VAN','MMS'),
        ('MMS','MMT'),
        ('MMT','MMP'),
        ('MMP','NOB'),
        ('ORL','VIR'),
        ('VIR','WSA'),
        ('WSA','OAK'),
        ('SAC','KCK'),
        ('KCK','KCO'),
        ('KCO','KCK'),
        ('KCK','CIN'),
        ('CIN','ROC'),
        ('CHP','BLB'),
        ('IND','INA'),
        ('INA','INO')]
st = set()
for t in new_team:
    st.add(t[0])
    st.add(t[1])

In [ ]:
import random
def load_roster(filename):
    base = json.load(open(filename,'rb'))

    
    for i in range(len(base['players'])):
        player = base['players'][i]
        if 'name' in player and player['name'] !='':
            sname = base['players'][i]['name'].split()
            base['players'][i]['firstName'] = sname[0]
            base['players'][i]['lastName'] = ' '.join(sname[1:])
            del base['players'][i]['name']
    players = base['players']
    base['version'] = 32

    return players,base
playersOG,base = load_roster('normed.json')
abbrev_to_tid = {_['abbrev']:_['tid'] for _ in base['teams']}
abbrev_to_tid['PHO'] = abbrev_to_tid['PHX']
abbrev_to_tid['CHO'] = abbrev_to_tid['CHA']
abbrev_to_tid['BRK'] = abbrev_to_tid['BKN']
abbrev_to_tid['NOH'] = abbrev_to_tid['NOP']
abbrev_to_tid['NJN'] = abbrev_to_tid['BKN']
abbrev_to_tid['SEA'] = abbrev_to_tid['OKC']
abbrev_to_tid['KCK'] = abbrev_to_tid['SAC']
abbrev_to_tid['SDC'] = abbrev_to_tid['LAC']
abbrev_to_tid['WSB'] = abbrev_to_tid['WAS']
abbrev_to_tid['CHH'] = abbrev_to_tid['CHA']
abbrev_to_tid['VAN'] = abbrev_to_tid['MEM']
abbrev_to_tid['NOK'] = abbrev_to_tid['NOP']
abbrev_to_tid['NOK'] = abbrev_to_tid['NOP']
abbrev_to_tid['BUF'] = abbrev_to_tid['LAC']
abbrev_to_tid['CAP'] = abbrev_to_tid['WAS']

for nt,ot in new_team:
    try:
        abbrev_to_tid[ot] = abbrev_to_tid[nt]
    except:
        print(nt,ot)

base['startingSeason'] = 2019
valid_tids = {abbrev_to_tid[_['abbrev']] for _ in base['teams']}

valid_teams = []
for t in base['teams']:
    if t['tid'] in valid_tids:
        tid = len(valid_teams)
        otid = t['tid']
        t['tid'] = tid
        for k,v in abbrev_to_tid.items():
            if v == otid:
                abbrev_to_tid[k] = tid
        valid_teams.append(t)

base['teams'] = valid_teams


del base['draftPicks']

In [ ]:
import random
team_ocs = defaultdict(list)

for p in players:
    name = p['name']
    sname = name.split(' ')
    p['firstName'] = sname[0]
    p['lastName'] = ' '.join(sname[1:])
    

    r = np.array([p['ratings'][0][inv_ymap[k]] for k in y_keys])
    ovr = (r * nc2).sum()
    
    p['ovr'] = ovr
    pt = p['abbrev']
    team_ocs[abbrev_to_tid[pt]].append(ovr)

    p['tid'] = abbrev_to_tid[pt]



In [ ]:
team_ocs = {k:sorted(v,reverse=True)[13] for k,v in team_ocs.items()}

In [ ]:
for player in players:
    if player['ovr'] < team_ocs[player['tid']]:
        player['tid'] = -1
    del player['ovr']

In [ ]:
base['players'] = players
with open('player_teams.json'.format(),'wt') as fp:
    json.dump(base,fp, sort_keys=True)